In [140]:
import os

In [141]:
%pwd

'/home/naveen/Naveen/Projects/Cat Or Dog Classificatoin/Cat-Or-Dog-Classification'

In [142]:


%cd /home/naveen/Naveen/Projects/Cat Or Dog Classificatoin/Cat-Or-Dog-Classification

/home/naveen/Naveen/Projects/Cat Or Dog Classificatoin/Cat-Or-Dog-Classification


In [143]:
%pwd

'/home/naveen/Naveen/Projects/Cat Or Dog Classificatoin/Cat-Or-Dog-Classification'

In [144]:


from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path



In [145]:
from Cat_Or_Dog_Classification.constants import *
from Cat_Or_Dog_Classification.utils.common import read_yaml, create_directories

In [146]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      


In [147]:


import os
import urllib.request as request
import zipfile
from Cat_Or_Dog_Classification import logger
from Cat_Or_Dog_Classification.utils.common import get_size

In [148]:


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [149]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-09-26 14:38:52,685: INFO: common: YAML file: config/config.yaml loaded successfully]
[2023-09-26 14:38:52,689: INFO: common: YAML file: params.yaml loaded successfully]
[2023-09-26 14:38:52,692: INFO: common: created directory at: artifacts]
[2023-09-26 14:38:52,699: INFO: common: created directory at: artifacts/data_ingestion]
[2023-09-26 14:39:01,467: INFO: 3126447047: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23481847
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "35e7d51459aecb822b112d65224c7ebf2329fdb3ea0358cae3c2eccca9d31d0f"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: EF70:216D92:242FA7:3C312A:65129FA4
Accept-Ranges: bytes
Date: Tue, 26 Sep 2023 09:08:54 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100020-HYD
X-Cache